In [3]:
from ultralytics import YOLO
import cv2
import time

def main():
    # Load model YOLOv11n
    model = YOLO("yolo11n.pt")  # hoặc "yolo11s.pt" nếu muốn model chính xác hơn

    # Mở camera (0 = webcam mặc định)
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Không thể mở webcam!")
        return

    prev_time = 0

    print("Đang chạy YOLOv11 realtime — nhấn 'q' để thoát.")
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Chạy dự đoán (inference)
        results = model(frame, verbose=False)

        # Vẽ bounding box trực tiếp
        annotated_frame = results[0].plot()

        # Tính FPS
        curr_time = time.time()
        fps = 1 / (curr_time - prev_time) if prev_time else 0
        prev_time = curr_time
        cv2.putText(annotated_frame, f"FPS: {fps:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Hiển thị ảnh có box
        cv2.imshow("YOLOv11 Camera", annotated_frame)

        # Thoát bằng phím q
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Đang chạy YOLOv11 realtime — nhấn 'q' để thoát.


In [19]:
from ultralytics import YOLO
import cv2
from pathlib import Path

# Danh sách các phương tiện giao thông
vehicles = [
    "car", "motorcycle", "bicycle", "bus", "truck",
    "train", "airplane", "boat", "ship", "scooter",
    "van", "helicopter"
]


def detect_image(model, image_path, save_result=False):
    """Nhận dạng và chỉ hiển thị các phương tiện giao thông"""
    image = cv2.imread(str(image_path))
    if image is None:
        print(f"Lỗi: Không thể đọc ảnh {image_path}")
        return

    # Chạy nhận dạng
    results = model(image, verbose=False)
    r = results[0]

    # Lấy thông tin box
    names = model.names  # tên các lớp
    boxes = r.boxes

    # Lọc đối tượng chỉ là phương tiện
    for box in boxes:
        cls_id = int(box.cls[0])
        conf = float(box.conf[0])
        label = names[cls_id]

        if label.lower() not in vehicles:
            continue  # bỏ qua nếu không phải phương tiện

        # Lấy tọa độ bbox
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
        cv2.putText(
            image,
            f"{label} {conf:.2f}",
            (x1, y1 - 10),
            cv2.FONT_HERSHEY_SIMPLEX,
            0.6,
            (0, 255, 0),
            2
        )

    # Hiển thị ảnh kết quả
    cv2.imshow("Vehicle Detection (YOLOv11)", image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

    if save_result:
        out_path = Path(image_path).stem + "_vehicles.jpg"
        cv2.imwrite(out_path, image)
        print(f"Đã lưu ảnh kết quả: {out_path}")


# --- Chạy thử ---
detect_image(YOLO("yolo11x.pt"), "image/download (2).jpg")


In [35]:
from ultralytics import YOLO
import cv2
import time
import math
import torch

# Danh sách phương tiện giao thông
vehicles = [
    "car", "motorcycle", "bicycle", "bus", "truck",
    "train", "airplane", "boat", "ship", "scooter",
    "van", "helicopter"
]

def detect_video(model, video_path=0, conf_thres=0.3):
    """Nhận dạng phương tiện giao thông trong video, tính vận tốc (px/s)"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    print(f"Sử dụng thiết bị: {device}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Không thể mở video: {video_path}")
        return
    
    prev_time = time.time()
    prev_positions = {}  # Lưu tâm bbox cũ của mỗi xe (index: class_id + vị trí)

    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    print("Đang chạy video... nhấn 'q' để thoát.")

    while True:
        ret, frame = cap.read()
        if not ret:
            break

        results = model(frame, conf=conf_thres, verbose=False)
        r = results[0]
        names = model.names
        boxes = r.boxes

        current_positions = {}

        for i, box in enumerate(boxes):
            cls_id = int(box.cls[0])
            label = names[cls_id].lower()
            conf = float(box.conf[0])

            if label not in vehicles or conf < conf_thres:
                continue

            # Tọa độ bounding box
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2  # tâm bbox

            # Tính vận tốc (nếu có dữ liệu trước)
            velocity = 0.0
            key = f"{cls_id}_{i}"  # tạm dùng index + class làm ID
            if key in prev_positions:
                px_prev, py_prev = prev_positions[key]
                dist = math.sqrt((cx - px_prev) ** 2 + (cy - py_prev) ** 2)
                velocity = dist * fps * 3.6  # pixel mỗi giây
            current_positions[key] = (cx, cy)

            # Vẽ bounding box
            cv2.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(
                frame,
                f"{label} v={velocity:.1f}km/h",
                (x1, y1 - 10),
                cv2.FONT_HERSHEY_SIMPLEX,
                0.5,
                (0, 0, 0),
                2
            )

        prev_positions = current_positions

        # Hiển thị FPS
        curr_time = time.time()
        fps_disp = 1 / (curr_time - prev_time) if prev_time else 0
        prev_time = curr_time
        cv2.putText(frame, f"FPS: {fps_disp:.1f}", (10, 30),
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)

        cv2.imshow("Vehicle Detection with Speed (YOLOv11)", frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()


# --- Chạy thử ---
if __name__ == "__main__":
    model = YOLO("yolo11x.pt")
    detect_video(model, "vidieo/2165-155327596.mp4")  # hoặc detect_video(model, 0) để dùng webcam


Sử dụng thiết bị: cuda
Đang chạy video... nhấn 'q' để thoát.


In [2]:
from ultralytics import YOLO
import cv2
import time
import math
import torch
import numpy as np
from collections import deque, defaultdict

# ===============================
# 🎛️ CẤU HÌNH DỄ CHỈNH SỬA
# ===============================

# Đường dẫn đến mô hình YOLO
MODEL_PATH = "yolo11s.pt"

# Đường dẫn video đầu vào (hoặc 0 cho webcam)
VIDEO_PATH = "vidieo/1900-151662242_small.mp4"

# Ngưỡng tin cậy cho phát hiện
CONF_THRESHOLD = 0.4

# Kích thước mục tiêu khi resize video (để tăng FPS)
TARGET_WIDTH = 1280

# Cấu hình Tracker
TRACKER_MAX_DISAPPEARED = 30   # Số frame cho phép mất trước khi xóa
TRACKER_MAX_DISTANCE = 100     # Khoảng cách tối đa để match object

# Hệ số chuyển đổi pixel → km/h (tuỳ theo video thực tế)
PIXEL_TO_KMH = 0.5

# FPS trung bình để làm mượt hiển thị
FPS_SMOOTHING_WINDOW = 30

# Danh sách phương tiện giao thông cần theo dõi
VEHICLES = [
    "car", "motorcycle", "bicycle", "bus", "truck",
    "train", "airplane", "boat", "ship", "scooter",
    "van", "helicopter"
]

# ===============================
# 🚗 CHƯƠNG TRÌNH CHÍNH
# ===============================

class VehicleTracker:
    """Tracker để theo dõi phương tiện với ID ổn định"""
    def __init__(self, max_disappeared=30, max_distance=100):
        self.next_id = 0
        self.objects = {}  # {id: (centroid, class_id, bbox)}
        self.disappeared = {}  # {id: số frame mất}
        self.velocities = defaultdict(lambda: deque(maxlen=10))  # Lưu lịch sử vận tốc
        self.max_disappeared = max_disappeared
        self.max_distance = max_distance
        
    def register(self, centroid, class_id, bbox):
        """Đăng ký object mới"""
        self.objects[self.next_id] = (centroid, class_id, bbox)
        self.disappeared[self.next_id] = 0
        self.next_id += 1
        
    def deregister(self, object_id):
        """Xóa object đã mất"""
        del self.objects[object_id]
        del self.disappeared[object_id]
        if object_id in self.velocities:
            del self.velocities[object_id]
    
    def update(self, detections):
        """Cập nhật tracker theo danh sách detection mới"""
        if len(detections) == 0:
            for object_id in list(self.disappeared.keys()):
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            return self.objects
        
        if len(self.objects) == 0:
            for detection in detections:
                self.register(*detection)
        else:
            object_ids = list(self.objects.keys())
            object_centroids = [self.objects[oid][0] for oid in object_ids]
            
            D = np.zeros((len(object_centroids), len(detections)))
            for i, obj_centroid in enumerate(object_centroids):
                for j, (det_centroid, _, _) in enumerate(detections):
                    D[i, j] = np.linalg.norm(np.array(obj_centroid) - np.array(det_centroid))
            
            rows = D.min(axis=1).argsort()
            cols = D.argmin(axis=1)[rows]
            
            used_rows, used_cols = set(), set()
            for row, col in zip(rows, cols):
                if row in used_rows or col in used_cols:
                    continue
                if D[row, col] > self.max_distance:
                    continue
                object_id = object_ids[row]
                self.objects[object_id] = detections[col]
                self.disappeared[object_id] = 0
                used_rows.add(row)
                used_cols.add(col)
            
            unused_rows = set(range(D.shape[0])) - used_rows
            for row in unused_rows:
                object_id = object_ids[row]
                self.disappeared[object_id] += 1
                if self.disappeared[object_id] > self.max_disappeared:
                    self.deregister(object_id)
            
            unused_cols = set(range(D.shape[1])) - used_cols
            for col in unused_cols:
                self.register(*detections[col])
        
        return self.objects

def detect_video(model, video_path=0, conf_thres=0.4):
    """Nhận dạng phương tiện giao thông với tracking và tính vận tốc ổn định"""
    device = "cuda" if torch.cuda.is_available() else "cpu"
    model.to(device)
    if device == "cuda":
        torch.backends.cudnn.benchmark = True
        print(f"✓ Sử dụng GPU: {torch.cuda.get_device_name(0)}")
    else:
        print("✓ Sử dụng CPU")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"❌ Không thể mở video: {video_path}")
        return
    
    fps = cap.get(cv2.CAP_PROP_FPS) or 30
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    if frame_width > TARGET_WIDTH:
        scale = TARGET_WIDTH / frame_width
        frame_width = TARGET_WIDTH
        frame_height = int(frame_height * scale)
    else:
        scale = 1.0
    
    print(f"✓ Video: {frame_width}x{frame_height} @ {fps:.1f} FPS")
    
    tracker = VehicleTracker(
        max_disappeared=int(fps if TRACKER_MAX_DISAPPEARED == 30 else TRACKER_MAX_DISAPPEARED),
        max_distance=TRACKER_MAX_DISTANCE
    )
    prev_positions = {}
    fps_deque = deque(maxlen=FPS_SMOOTHING_WINDOW)
    prev_time = time.time()
    
    print("🎬 Đang chạy... nhấn 'q' để thoát")

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        if scale != 1.0:
            frame = cv2.resize(frame, (frame_width, frame_height))
        
        with torch.no_grad():
            results = model(frame, conf=conf_thres, verbose=False, half=(device == "cuda"))
        
        r = results[0]
        names = model.names
        boxes = r.boxes
        detections = []
        for box in boxes:
            cls_id = int(box.cls[0])
            label = names[cls_id].lower()
            conf = float(box.conf[0])
            if label not in VEHICLES or conf < conf_thres:
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
            bbox = (x1, y1, x2, y2)
            detections.append(((cx, cy), cls_id, bbox))
        
        tracked_objects = tracker.update(detections)
        
        for object_id, (centroid, cls_id, bbox) in tracked_objects.items():
            cx, cy = centroid
            x1, y1, x2, y2 = bbox
            label = names[cls_id].lower()
            velocity_kmh = 0.0
            if object_id in prev_positions:
                px_prev, py_prev = prev_positions[object_id]
                dist_pixels = math.sqrt((cx - px_prev)**2 + (cy - py_prev)**2)
                velocity_px_per_sec = dist_pixels * fps
                velocity_kmh = velocity_px_per_sec * PIXEL_TO_KMH
                tracker.velocities[object_id].append(velocity_kmh)
                velocity_kmh = np.mean(list(tracker.velocities[object_id]))
            prev_positions[object_id] = (cx, cy)
            
            color = (0, 255, 0) if velocity_kmh < 40 else (0, 165, 255) if velocity_kmh < 80 else (0, 0, 255)
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            text = f"ID:{object_id} {label}"
            speed_text = f"{velocity_kmh:.1f} km/h"
            (tw, th), _ = cv2.getTextSize(text, cv2.FONT_HERSHEY_SIMPLEX, 0.5, 1)
            cv2.rectangle(frame, (x1, y1-th-25), (x1+tw, y1), color, -1)
            cv2.putText(frame, text, (x1, y1-15), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(frame, speed_text, (x1, y1-2), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.circle(frame, (cx, cy), 4, color, -1)
        
        curr_time = time.time()
        fps_value = 1 / (curr_time - prev_time + 1e-6)
        fps_deque.append(fps_value)
        avg_fps = np.mean(fps_deque)
        prev_time = curr_time
        
        info_bg = np.zeros((80, 300, 3), dtype=np.uint8)
        cv2.putText(info_bg, f"FPS: {avg_fps:.1f}", (10, 25), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
        cv2.putText(info_bg, f"Tracked: {len(tracked_objects)}", (10, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 1)
        frame[10:90, 10:310] = cv2.addWeighted(frame[10:90, 10:310], 0.3, info_bg, 0.7, 0)
        
        cv2.imshow("Vehicle Detection & Tracking (YOLOv11)", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("✓ Hoàn thành!")


if __name__ == "__main__":
    model = YOLO(MODEL_PATH)
    detect_video(model, VIDEO_PATH, conf_thres=CONF_THRESHOLD)


✓ Sử dụng GPU: NVIDIA GeForce GTX 1650
✓ Video: 1280x720 @ 25.0 FPS
🎬 Đang chạy... nhấn 'q' để thoát
✓ Hoàn thành!
